# Phase 2A: 청킹 전략 실험 노트북

이 노트북은 원본 소스 데이터를 효과적으로 청킹하기 위한 다양한 전략을 실험합니다.

## 목표
1. 데이터 소스별 최적 청킹 전략 도출
2. Chunk Size와 Overlap 파라미터 튜닝
3. 검색 품질 비교 (Relevance, Coverage)

## 실험 대상 소스
- 미쉐린 가이드 리뷰 (200-500자)
- 블루리본 리뷰 (200-500자)
- 셰프 인터뷰 (1000-3000자)
- 코스 구성 설명 (300-800자)
- 브랜드 철학/공간 (500-1000자)

## 청킹 방법
1. **Document 방식**: 분할 없이 전체 문서를 하나의 청크로
2. **Recursive 방식**: 문단 → 문장 순으로 재귀적 분할
3. **Semantic 방식**: 의미 기반 분할 (spaCy)

## 실험 파라미터
- Chunk Size: 400, 512, 600, 800, 1200 (토큰/문자 기준)
- Overlap: 0, 50, 100, 200, 300 (문자 기준)

## 섹션 1: 패키지 설치

In [ ]:
# 기존 패키지 + 청킹용 추가 패키지
%pip install google-cloud-aiplatform google-genai python-dotenv chromadb langchain langchain-text-splitters

## 섹션 2: 라이브러리 Import

In [ ]:
import os
import json
import pandas as pd
import numpy as np
import vertexai
from google import genai
from google.genai import types
from dotenv import load_dotenv
import chromadb
from datetime import datetime
from vertexai.language_models import TextEmbeddingModel
from langchain.text_splitter import RecursiveCharacterTextSplitter
import time

print("✅ 라이브러리 Import 완료")

## 섹션 3: Vertex AI 초기화

In [ ]:
load_dotenv()

PROJECT_ID = "wad-dw"
LOCATION = "us-central1"

vertexai.init(project=PROJECT_ID, location=LOCATION)
client = genai.Client(vertexai=True, project=PROJECT_ID, location=LOCATION)

print(f"✅ Vertex AI 초기화 완료: {PROJECT_ID} ({LOCATION})")

## 섹션 4: 샘플 데이터 준비

실제 매장 데이터를 샘플로 사용합니다.

In [ ]:
# 샘플 데이터: 다양한 소스 타입별 텍스트
SAMPLE_DATA = {
    "michelin_review": {
        "shop_name": "모수",
        "text": """모수는 안성재 셰프가 운영하는 모던 한식 레스토랑입니다. 화덕 구이 기법을 활용한 창의적인 한식을 선보이며, 제철 식재료를 중심으로 한국의 전통 조리법에 현대적 감각을 더했습니다. 깔끔하고 모던한 공간에서 셰프의 철학이 담긴 정교한 플레이팅을 경험할 수 있습니다. 미쉐린 2스타를 유지하며 한국 파인다이닝의 새로운 기준을 제시하고 있습니다.""",
        "category": "fine_dining"
    },
    "chef_interview": {
        "shop_name": "밍글스",
        "text": """Q: 밍글스의 요리 철학은 무엇인가요?
A: 밍글스는 '한국의 맛을 현대적으로 재해석한다'는 철학을 가지고 있습니다. 한국 전통 식재료와 발효 기법을 활용하되, 세계적인 조리 기법과 플레이팅으로 표현합니다. 예를 들어 장류, 김치, 젓갈 같은 한국 고유의 발효 식품을 소스나 조미료로 활용하여 깊은 감칠맛을 냅니다.

Q: 가장 자랑하는 시그니처 메뉴는?
A: '장trio'가 대표적입니다. 된장, 간장, 고추장을 각각 다른 방식으로 활용한 세 가지 요리를 한 접시에 담아냅니다. 한국인에게 익숙한 장맛을 전혀 새로운 방식으로 경험하게 하죠. 또한 '지짐이 구이'는 전통 한식의 지짐이를 고급 다이닝 스타일로 재해석한 메뉴로, 해외 손님들에게 특히 인기가 많습니다.

Q: 식재료 소싱은 어떻게 하시나요?
A: 매일 아침 노량진 수산시장과 가락시장에 직접 가서 제철 식재료를 선별합니다. 특히 지방의 소규모 농가와 직거래하며 유기농 채소를 공급받고 있습니다. 한우는 횡성, 해산물은 남해안 어촌, 쌀은 이천에서 직접 공수합니다.

Q: 밍글스를 찾는 고객들에게 전하고 싶은 메시지는?
A: 밍글스는 단순히 음식을 먹는 곳이 아니라, 한국 문화를 경험하는 공간입니다. 각 요리마다 한국의 역사와 이야기가 담겨 있고, 저희는 그것을 현대적 언어로 번역해서 전달합니다. 한국인에게는 익숙한 맛의 새로운 발견을, 외국인에게는 한국 미식의 진수를 느끼셨으면 합니다.""",
        "category": "fine_dining"
    },
    "course_description": {
        "shop_name": "권숙수",
        "text": """권숙수의 코스는 한국의 사계절을 담은 12가지 요리로 구성됩니다. 

1. 전채: 제철 채소를 활용한 계절 전채 3종 (봄나물, 여름 열매, 가을 버섯 등)
2. 어패류: 제주산 전복과 남해안 활어를 활용한 숙회 또는 찜
3. 탕: 전통 육수를 베이스로 한 맑은 탕 (소뼈 우린 육수, 닭 육수 등)
4. 구이: 한우 등심 숯불 구이 또는 제철 생선 구이
5. 조림: 전통 조림 기법을 활용한 메인 요리
6. 식사: 돌솥밥과 김치, 장아찌, 된장찌개
7. 후식: 전통 한과와 제철 과일

모든 코스는 권우중 셰프가 직접 선별한 제철 식재료로 구성되며, 한국 전통 조리법을 현대적으로 해석합니다.""",
        "category": "fine_dining"
    },
    "brand_philosophy": {
        "shop_name": "나베노이즘",
        "text": """나베노이즘은 '스시의 본질'을 추구하는 오마카세 레스토랑입니다. 이시카와 히데아키 셰프는 도쿄에서 15년간 스시 장인으로 수련한 후, 서울에서 정통 에도마에 스시를 선보이고 있습니다.

우리의 철학은 세 가지 핵심 가치로 요약됩니다:

1. 식재료의 본질: 매일 새벽 노량진 수산시장과 일본 츠키지 시장에서 최고 등급의 수산물을 공수합니다. 제철 생선만을 사용하며, 각 어종의 최적 숙성 기간을 지켜 감칠맛을 극대화합니다.

2. 전통 기법의 고수: 샤리(초밥의 밥)는 아키타현 고시히카리 쌀을 사용하며, 적미초를 직접 만들어 간을 맞춥니다. 네타(스시 재료)는 전통 에도마에 방식으로 손질하고, 100년 전통의 조림 기법(니키리, 츠메)을 그대로 재현합니다.

3. 장인정신: 하루 두 타임, 최대 16명의 손님만 받으며, 셰프가 직접 한 점 한 점 쥐어 드립니다. 고객과의 대화를 통해 취향을 파악하고, 그에 맞춰 코스를 조정합니다.

나베노이즘의 공간은 미니멀하고 절제된 일본식 미학을 추구합니다. 히노키 원목 카운터와 계절을 담은 이케바나, 그리고 조용히 흐르는 재즈 음악이 어우러져 스시 본연의 맛에 집중할 수 있는 환경을 제공합니다.""",
        "category": "fine_dining"
    }
}

print("✅ 샘플 데이터 준비 완료")
print(f"   - 미쉐린 리뷰: {len(SAMPLE_DATA['michelin_review']['text'])}자")
print(f"   - 셰프 인터뷰: {len(SAMPLE_DATA['chef_interview']['text'])}자")
print(f"   - 코스 설명: {len(SAMPLE_DATA['course_description']['text'])}자")
print(f"   - 브랜드 철학: {len(SAMPLE_DATA['brand_philosophy']['text'])}자")

## 섹션 5: 청킹 함수 정의

In [ ]:
# ========================================
# 청킹 함수
# ========================================

def chunk_document(text, method="document", chunk_size=512, chunk_overlap=0):
    """
    텍스트를 주어진 방식으로 청킹합니다.
    
    Args:
        text: 원본 텍스트
        method: 청킹 방식 ("document", "recursive", "semantic")
        chunk_size: 청크 크기 (문자 수)
        chunk_overlap: 오버랩 크기 (문자 수)
    
    Returns:
        청크 리스트
    """
    
    if method == "document":
        # 전체 문서를 하나의 청크로
        return [text]
    
    elif method == "recursive":
        # LangChain RecursiveCharacterTextSplitter 사용
        splitter = RecursiveCharacterTextSplitter(
            chunk_size=chunk_size,
            chunk_overlap=chunk_overlap,
            separators=["\n\n", "\n", ". ", "? ", "! ", " ", ""],
            length_function=len
        )
        return splitter.split_text(text)
    
    elif method == "semantic":
        # 문장 단위로 분할 (간단한 구현)
        # 실제로는 spaCy를 사용할 수 있지만, 여기서는 간단히 구현
        sentences = text.replace(". ", ".\n").replace("? ", "?\n").replace("! ", "!\n").split("\n")
        
        chunks = []
        current_chunk = []
        current_length = 0
        
        for sent in sentences:
            sent = sent.strip()
            if not sent:
                continue
            
            if current_length + len(sent) > chunk_size and current_chunk:
                # 청크 저장
                chunks.append(" ".join(current_chunk))
                
                # 오버랩: 마지막 문장 유지
                if chunk_overlap > 0 and current_chunk:
                    overlap_text = current_chunk[-1]
                    current_chunk = [overlap_text, sent]
                    current_length = len(overlap_text) + len(sent)
                else:
                    current_chunk = [sent]
                    current_length = len(sent)
            else:
                current_chunk.append(sent)
                current_length += len(sent)
        
        # 마지막 청크
        if current_chunk:
            chunks.append(" ".join(current_chunk))
        
        return chunks
    
    else:
        raise ValueError(f"Unknown method: {method}")


def generate_embedding(text, model="text-embedding-004"):
    """텍스트를 768차원 벡터로 변환"""
    try:
        embedding_model = TextEmbeddingModel.from_pretrained(model)
        embeddings = embedding_model.get_embeddings([text])
        return embeddings[0].values
    except Exception as e:
        print(f"❌ 임베딩 생성 실패: {e}")
        return None


def analyze_chunks(chunks, original_text):
    """
    청크 분석 통계
    """
    chunk_lengths = [len(c) for c in chunks]
    
    return {
        "num_chunks": len(chunks),
        "original_length": len(original_text),
        "avg_chunk_length": np.mean(chunk_lengths),
        "min_chunk_length": np.min(chunk_lengths),
        "max_chunk_length": np.max(chunk_lengths),
        "std_chunk_length": np.std(chunk_lengths)
    }


print("✅ 청킹 함수 정의 완료")
print("   - chunk_document(): 텍스트를 청킹")
print("   - analyze_chunks(): 청크 통계 분석")
print("   - generate_embedding(): 임베딩 생성")

## 섹션 6: 청킹 전략 비교 실험

다양한 소스 타입에 대해 여러 청킹 설정을 비교합니다.

In [ ]:
# ========================================
# 실험 설정
# ========================================

# 테스트할 청킹 설정
CHUNKING_CONFIGS = {
    "no_split": {
        "method": "document",
        "chunk_size": None,
        "chunk_overlap": 0,
        "description": "분할 없음 (전체 문서)"
    },
    "small_no_overlap": {
        "method": "recursive",
        "chunk_size": 400,
        "chunk_overlap": 0,
        "description": "작은 청크, 오버랩 없음"
    },
    "medium_small_overlap": {
        "method": "recursive",
        "chunk_size": 512,
        "chunk_overlap": 50,
        "description": "중간 청크, 작은 오버랩"
    },
    "medium_medium_overlap": {
        "method": "recursive",
        "chunk_size": 600,
        "chunk_overlap": 100,
        "description": "중간 청크, 중간 오버랩"
    },
    "large_large_overlap": {
        "method": "recursive",
        "chunk_size": 800,
        "chunk_overlap": 200,
        "description": "큰 청크, 큰 오버랩 (25%)"
    },
    "semantic_medium": {
        "method": "semantic",
        "chunk_size": 512,
        "chunk_overlap": 50,
        "description": "의미 기반 청킹"
    }
}

# 실험 결과 저장
experiment_results = []

print("🔬 청킹 전략 비교 실험 시작\n")
print("=" * 100)

# 각 소스 타입별로 실험
for source_type, source_data in SAMPLE_DATA.items():
    text = source_data["text"]
    shop_name = source_data["shop_name"]
    
    print(f"\n📋 소스: {source_type} ({shop_name})")
    print(f"   원본 길이: {len(text)}자")
    print("─" * 100)
    
    # 각 청킹 설정 테스트
    for config_name, config in CHUNKING_CONFIGS.items():
        print(f"\n   [{config_name}] {config['description']}")
        
        # 청킹 실행
        chunks = chunk_document(
            text=text,
            method=config["method"],
            chunk_size=config.get("chunk_size", 512),
            chunk_overlap=config.get("chunk_overlap", 0)
        )
        
        # 통계 분석
        stats = analyze_chunks(chunks, text)
        
        # 결과 저장
        result = {
            "source_type": source_type,
            "shop_name": shop_name,
            "config_name": config_name,
            "method": config["method"],
            "chunk_size": config.get("chunk_size"),
            "chunk_overlap": config.get("chunk_overlap"),
            **stats
        }
        experiment_results.append(result)
        
        # 출력
        print(f"      청크 개수: {stats['num_chunks']}개")
        print(f"      평균 길이: {stats['avg_chunk_length']:.1f}자")
        print(f"      범위: {stats['min_chunk_length']:.0f} ~ {stats['max_chunk_length']:.0f}자")
        
        # 첫 번째 청크 미리보기
        if chunks:
            preview = chunks[0][:100] + "..." if len(chunks[0]) > 100 else chunks[0]
            print(f"      미리보기: {preview}")

print("\n" + "=" * 100)
print(f"\n✅ 실험 완료: {len(experiment_results)}개 설정 테스트")

## 섹션 7: 실험 결과 분석

In [ ]:
# DataFrame으로 변환
df_results = pd.DataFrame(experiment_results)

print("📊 청킹 실험 결과 요약\n")
print("=" * 100)

# 소스 타입별 요약
for source_type in SAMPLE_DATA.keys():
    print(f"\n📌 {source_type}")
    print("─" * 100)
    
    df_source = df_results[df_results['source_type'] == source_type]
    
    # 테이블 출력
    display_cols = ['config_name', 'num_chunks', 'avg_chunk_length', 'min_chunk_length', 'max_chunk_length']
    print(df_source[display_cols].to_string(index=False))

print("\n" + "=" * 100)

# CSV 저장
csv_filename = f"chunking_experiment_results_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
df_results.to_csv(csv_filename, index=False, encoding='utf-8-sig')
print(f"\n💾 결과 저장: {csv_filename}")

## 섹션 8: 권장 청킹 전략

실험 결과를 바탕으로 소스 타입별 최적 청킹 전략을 제안합니다.

In [ ]:
# ========================================
# 권장 청킹 설정
# ========================================

RECOMMENDED_CHUNKING = {
    "michelin_review": {
        "method": "document",
        "chunk_size": None,
        "chunk_overlap": 0,
        "reason": "짧고 완결된 텍스트 (200-500자). 분할 시 맥락 손실 우려."
    },
    "blueribbon_review": {
        "method": "document",
        "chunk_size": None,
        "chunk_overlap": 0,
        "reason": "미쉐린과 동일. 짧고 완결된 리뷰 단위."
    },
    "chef_interview": {
        "method": "recursive",
        "chunk_size": 800,
        "chunk_overlap": 200,
        "reason": "긴 텍스트 (1000-3000자). Q&A 맥락 유지를 위해 큰 청크 + 25% 오버랩."
    },
    "course_description": {
        "method": "semantic",
        "chunk_size": 512,
        "chunk_overlap": 50,
        "reason": "중간 길이 (300-800자). 요리별 설명 단위로 분할."
    },
    "brand_philosophy": {
        "method": "recursive",
        "chunk_size": 600,
        "chunk_overlap": 100,
        "reason": "중간~긴 텍스트 (500-1000자). 철학/공간 맥락 유지."
    }
}

print("💡 소스 타입별 권장 청킹 전략\n")
print("=" * 100)

for source_type, config in RECOMMENDED_CHUNKING.items():
    print(f"\n📌 {source_type}")
    print("─" * 100)
    print(f"   방식: {config['method']}")
    print(f"   청크 크기: {config['chunk_size'] if config['chunk_size'] else 'N/A (전체)'}")
    print(f"   오버랩: {config['chunk_overlap']}자")
    print(f"   이유: {config['reason']}")

print("\n" + "=" * 100)

# JSON으로 저장
config_filename = "recommended_chunking_config.json"
with open(config_filename, 'w', encoding='utf-8') as f:
    json.dump(RECOMMENDED_CHUNKING, f, ensure_ascii=False, indent=2)

print(f"\n💾 설정 저장: {config_filename}")
print("\n✅ 다음 단계: 이 설정을 source_indexer.ipynb에서 활용합니다.")

## 섹션 9: 벡터 DB 검색 품질 테스트 (선택)

실제로 청킹 전략이 검색 품질에 미치는 영향을 테스트합니다.

In [ ]:
# 임시 Chroma 컬렉션 생성 및 검색 품질 테스트
print("🔍 벡터 DB 검색 품질 테스트 (선택 사항)\n")
print("이 섹션은 실제 검색 품질을 측정하고 싶을 때 실행하세요.")
print("시간이 오래 걸릴 수 있습니다 (임베딩 생성).")
print("\n⚠️ 이 섹션을 건너뛰고 싶다면 다음 셀로 이동하세요.")

In [ ]:
# 검색 품질 테스트 (옵션)
# 주의: 이 코드는 많은 API 호출을 발생시킵니다

# 테스트 쿼리
TEST_QUERIES = [
    "셰프의 조리 철학과 식재료 소싱 방법",
    "코스 구성과 시그니처 메뉴",
    "전통 기법과 현대적 해석"
]

# 이 부분은 실제로 구현하고 싶을 때만 실행
# (임베딩 생성 비용이 발생하므로 주석 처리)
print("⚠️ 검색 품질 테스트는 현재 구현되지 않았습니다.")
print("   필요 시 source_indexer.ipynb에서 구현할 예정입니다.")

## 결론

이 노트북에서 다양한 청킹 전략을 실험했습니다.

### 다음 단계
1. **Phase 2B**: `source_collector.ipynb` - 실제 원본 소스 데이터 수집
2. **Phase 2C**: `source_indexer.ipynb` - 권장 설정으로 벡터 DB 인덱싱
3. **Phase 3**: `main_rag_source.ipynb` - Source-based RAG 구현
4. **Phase 4**: `main_rag_hybrid.ipynb` - Hybrid RAG 구현